In [49]:
# Example English to Myanmar (Burmese) translation dataset
english_sentences = [
    "I love Burmese food.",
    "She speaks English fluently.",
    "They are going to the market."
]

burmese_sentences = [
    "<start> မြန်မာ အစားအသောက်ကို ကြိုက်တယ် <end>",
    "<start> သူမက အင်္ဂလိပ် စကားကို ကောင်းကောင်း ပြောနိုင်တယ်။ <end>",
    "<start> သူတို့ စျေးသွားကြတယ်။ <end>"
]


In [50]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense # type: ignore
from tensorflow.keras.preprocessing.text import Tokenizer # type: ignore
from tensorflow.keras.preprocessing.sequence import pad_sequences # type: ignore

In [51]:
# Tokenize English sentences
en_tokenizer = Tokenizer()
en_tokenizer.fit_on_texts(english_sentences)
english_sequences = en_tokenizer.texts_to_sequences(english_sentences)

# Tokenize Burmese sentences
my_tokenizer = Tokenizer(filters='')
my_tokenizer.fit_on_texts(burmese_sentences)
burmese_sequences = my_tokenizer.texts_to_sequences(burmese_sentences)

en_vocab_size = len(en_tokenizer.word_index) + 1
my_vocab_size = len(my_tokenizer.word_index) + 1

# Pad sequences to ensure uniform length
max_seq_length = max(max(len(seq) for seq in english_sequences), max(len(seq) for seq in burmese_sequences))


In [52]:
encoder_input_data = pad_sequences(english_sequences, maxlen=max_seq_length, padding='post')
decoder_input_data = pad_sequences(burmese_sequences, maxlen=max_seq_length, padding='post')
decoder_target_data = np.zeros_like(decoder_input_data)
decoder_target_data[:, :-1] = decoder_input_data[:, 1:]

In [53]:
# Encoder model
encoder_inputs = Input(shape=(max_seq_length,))
encoder_embedding = Embedding(en_vocab_size, 256)(encoder_inputs)
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder model
decoder_inputs = Input(shape=(max_seq_length,))
decoder_embedding = Embedding(my_vocab_size, 256)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(my_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)



In [54]:
# Define the seq2seq model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Train the model
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=32,
          epochs=10,
          validation_split=0.2)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 2.5608 - val_loss: 2.5275
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 2.5250 - val_loss: 2.4944
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 2.4859 - val_loss: 2.4540
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 2.4382 - val_loss: 2.4015
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 2.3757 - val_loss: 2.3310
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 2.2898 - val_loss: 2.2345
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 2.1707 - val_loss: 2.1008
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 2.0155 - val_loss: 1.9178
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 1.8702 - val_loss: 1.7135
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.8561 - val_loss: 1.6391


In [55]:
# Encoder model
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder model
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)


In [56]:
# Function to perform translation
def translate(input_text):
    input_seq = en_tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=max_seq_length, padding='post')
    
    states_value = encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = my_tokenizer.word_index['<start>']  # Start token
    
    translated_sentence = []
    for _ in range(max_seq_length):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = my_tokenizer.index_word.get(sampled_token_index, '<unk>')
        translated_sentence.append(sampled_word)
        
        if sampled_word == '<end>':
            break
        
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        
        states_value = [h, c]
    
    return ' '.join(translated_sentence)

# Example translation
input_text = "I love Burmese food."
translated_text = translate(input_text)
print("Input:", input_text)
print("Translated:", translated_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Input: I love Burmese food.
Translated: <unk> <unk> <unk> <unk> <unk> <unk> <unk>
